# House price model training

Training of the final house price model based on England-wide training data.

In [1]:
import geopandas as gpd
import numpy as np
import pickle
import libpysal
import pandas as pd

from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict

/var/folders/2f/fhks6w_d0k556plcv3rfmshw0000gn/T/ipykernel_509/1880529722.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
data_folder = "/Users/martin/Library/CloudStorage/OneDrive-SharedLibraries-TheAlanTuringInstitute/Daniel Arribas-Bel - demoland_data"

Load the data

In [3]:
data = gpd.read_parquet( f"{data_folder}/processed/oa_data_england.parquet")

Filter only explanatory variables.

In [4]:
exvars = data.drop(
    columns=[
        "geometry",
        "air_quality",
        "house_price",
    ]
)

Create weights of the order 5 identified as optimal.

In [9]:
queen = libpysal.weights.Queen.from_dataframe(data)
queen5 = libpysal.weights.higher_order(queen, k=5, lower_order=True)

/Users/martin/mambaforge/envs/demoland/lib/python3.11/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 51 disconnected components.
 There are 19 islands with ids: 1676, 2132, 3036, 21306, 33133, 34428, 42635, 42654, 68863, 72800, 74393, 105153, 108399, 134057, 140649, 141143, 141475, 144653, 149708.
  warnings.warn(message)


Compute spatial lag.

In [10]:
queen5.transform = "r"
for col in exvars.columns.copy():
    exvars[f"{col}_lag"] = libpysal.weights.spatial_lag.lag_spatial(queen5, exvars[col])

('WARNING: ', 1676, ' is an island (no neighbors)')
('WARNING: ', 2132, ' is an island (no neighbors)')
('WARNING: ', 3036, ' is an island (no neighbors)')
('WARNING: ', 21306, ' is an island (no neighbors)')
('WARNING: ', 33133, ' is an island (no neighbors)')
('WARNING: ', 34428, ' is an island (no neighbors)')
('WARNING: ', 42635, ' is an island (no neighbors)')
('WARNING: ', 42654, ' is an island (no neighbors)')
('WARNING: ', 68863, ' is an island (no neighbors)')
('WARNING: ', 72800, ' is an island (no neighbors)')
('WARNING: ', 74393, ' is an island (no neighbors)')
('WARNING: ', 105153, ' is an island (no neighbors)')
('WARNING: ', 108399, ' is an island (no neighbors)')
('WARNING: ', 134057, ' is an island (no neighbors)')
('WARNING: ', 140649, ' is an island (no neighbors)')
('WARNING: ', 141143, ' is an island (no neighbors)')
('WARNING: ', 141475, ' is an island (no neighbors)')
('WARNING: ', 144653, ' is an island (no neighbors)')
('WARNING: ', 149708, ' is an island (no n

Create mask to ignore missing values in training.

In [11]:
mask = data.house_price.notna()

Initialise the model.

In [12]:
regressor = HistGradientBoostingRegressor(
    random_state=0, max_bins=128, max_iter=1000
)


Train the model.

In [14]:
regressor.fit(exvars[mask], np.log(data.house_price[mask]))

HistGradientBoostingRegressor(max_bins=128, max_iter=1000, random_state=0)

Test the prediction.

In [15]:
regressor.predict(exvars.iloc[:10])

array([7.43681357, 7.52412204, 7.34451913, 7.64584292, 7.45234704,
       7.7307261 , 7.74621834, 7.5703991 , 7.53491617, 7.4739106 ])

Save to file.

In [ ]:
with open(f"{data_folder}/models/house_price_predictor_england_no_london.pickle", "wb") as f:
    pickle.dump(regressor, f)